# Ekhator Uwaila's Notebook
# Assignment 2 - Elementary Probability and Information Theory 
# Boise State University NLP - Dr. Kennington

### Instructions and Hints:

* This notebook loads some data into a `pandas` dataframe, then does a small amount of preprocessing. Make sure your data can load by stepping through all of the cells up until question 1. 
* Most of the questions require you to write some code. In many cases, you will write some kind of probability function like we did in class using the data. 
* Some of the questions only require you to write answers, so be sure to change the cell type to markdown or raw text
* Don't worry about normalizing the text this time (e.g., lowercase, etc.). Just focus on probabilies. 
* Most questions can be answered in a single cell, but you can make as many additional cells as you need. 
* Follow the instructions on the corresponding assignment Trello card for submitting your assignment. 

In [67]:
import pandas as pd 

data = pd.read_csv('pnp-train.txt',delimiter='\t',encoding='latin-1', # utf8 encoding didn't work for this
                  names=['type','name']) # supply the column names for the dataframe

# this next line creates a new column with the lower-cased first word
data['first_word'] = data['name'].map(lambda x: x.lower().split()[0])

In [68]:
data[:10]

,type,name,first_word
0,drug,Dilotab,dilotab
1,movie,Beastie Boys: Live in Glasgow,beastie
2,person,Michelle Ford-Eriksson,michelle
3,place,Ramsbury,ramsbury
4,place,Market Bosworth,market
5,drug,Cyanide Antidote Package,cyanide
6,person,Bill Johnson,bill
7,place,Ettalong,ettalong
8,movie,The Suicide Club,the
9,place,Pézenas,pézenas


In [69]:
data.describe()

,type,name,first_word
count,21001,21001,21001
unique,5,20992,13703
top,movie,Epinal,the
freq,6262,2,635


## 1. Write a probability function/distribution $P(T)$ over the types. 

Hints:

* The Counter library might be useful: `from collections import Counter`
* Write a function `def P(T='')` that returns the probability of the specific value for T
* You can access the types from the dataframe by calling `data['type']`

In [70]:
#Importing the counter library
from collections import Counter as ctr

In [71]:
type_ctr = ctr(data.type)
def P(T=''):
    """ Calculate the probability of each type in the dataset """
    if T not in type_ctr: return 0.0001
    return type_ctr[T] / len(data)

## 2. What is `P(T='movie')` ?

In [72]:
P('movie')

0.29817627732012764

## 3. Show that your probability distribution sums to one.

In [73]:
#Showing that the probability distributions sum to one manually
P('movie') + P('drug') + P('person') + P('place') + P('company')

1.0

In [74]:
#Showing that the probability distributions sum to one using an inline for loop
#sum(P(T) for T in data['type'].unique())
sum(P(T) for T in ctr(data.type))

1.0

## 4. Write a joint distribution using the type and the first word of the name

Hints:

* The function is $P2(T,W_1)$
* You will need to count up types AND the first words, for example: ('person','bill)
* Using the [itertools.product](https://docs.python.org/2/library/itertools.html#itertools.product) function was useful for me here

In [75]:
fword_ctr = ctr(data.first_word)
def P2(T='', W1=''):
    """ Calculate the joint probability of a type and firstword in the dataset """
    joint = len(data[(data['type'] == T ) & (data['first_word'] == W1)])
    jointd = joint / len(data)
    if T not in type_ctr: return 0.0001
    if W1 not in fword_ctr : return 0.0001
    return jointd
    

## 5. What is P2(T='person', W1='bill')? What about P2(T='movie',W1='the')?

In [76]:
P2(T='person', W1='bill')

0.00047616780153326033

In [77]:
P2(T='movie', W1='the')

0.02747488214846912

## 6. Show that your probability distribution P(T,W1) sums to one.

In [78]:
#sum(P(T) for T in data['type'].unique())
sum(P2(T, W1) for T in type_ctr for W1 in fword_ctr)

0.9999999999997995

## 7. Make a new function Q(T) from marginalizing over P(T,W1) and make sure that Q(T) sums to one.

Hints:

* Your Q function will call P(T,W1)
* Your check for the sum to one should be the same answer as Question 3, only it calls Q instead of P.

In [79]:
def Q(T):
    """ Calculate the marginal probability of a type in the dataset """
    margProb = sum(P2(T, W1) for W1 in fword_ctr)
    if T not in type_ctr: return 0.0001
    return margProb


In [80]:
#Showing that Q(t) sums to one using an inline for loop

sum(Q(T) for T in ctr(data.type))

1.0000000000000266

In [81]:
Q('movie')

0.29817627732011875

## 8. What is the KL Divergence of your Q function and your P function for Question 1?

* Even if you know the answer, you still need to write code that computes it.

In [82]:
import math
(P('movie') * math.log(P('movie') / Q('movie')) + P('person') * math.log(P('person') / Q('person')) + 
 P('place') * math.log(P('place') / Q('place')) + P('drug') * math.log(P('drug') / Q('drug')) + P('company') * math.log(P('company') / Q('company')))

-2.6643941088945053e-14

In [83]:
#Creating a function to calculate KL Divergence
def KL(T):
    kl = P(T) * math.log(P(T)/Q(T))
    return kl

In [84]:
sum(KL(T) for T in type_ctr)

-2.6643941088945053e-14

## 9. Convert from P(T,W1) to P(W1|T) 

Hints:

* Just write a comment cell, no code this time. 
* Note that $P(T,W1) = P(W1,T)$

Answer: The conditional probability of W1 given T is, $P(W1|T) = \frac{P(W1,T)}{P(T)}$ 

## 10. Write a function `Pwt` (that calls the functions you already have) to compute $P(W_1|T)$.

* This will be something like the multiplication rule, but you may need to change something

In [85]:
def Pwt(W1='',T=''):
    return P2(T,W1)/P(T)

## 11. What is P(W1='the'|T='movie')?

In [86]:
Pwt(W1='the',T='movie')

0.09214308527626956

## 12. Use Baye's rule to convert from P(W1|T) to P(T|W1). Write a function Ptw to reflect this. 

Hints:

* Call your other functions.
* You may need to write a function for P(W1) and you may need a new counter for `data['first_word']`

In [87]:
def Pw(W1=''):
    """ Calculate the probability of each first word in the dataset """
    if W1 not in fword_ctr: return 0.0001
    return fword_ctr[W1] / len(data)

In [88]:
def Ptw(T='',W1=''):
    return (Pwt(W1,T) * P(T)) / Pw(W1)

## 13 
### What is P(T='movie'|W1='the')? 
### What about P(T='person'|W1='the')?
### What about P(T='drug'|W1='the')?
### What about P(T='place'|W1='the')
### What about P(T='company'|W1='the')

In [89]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [90]:
Ptw(T='person',W1='the')

0.0

In [91]:
Ptw(T='drug',W1='the')

0.0

In [92]:
Ptw(T='place',W1='the')

0.0015748031496062992

In [93]:
Ptw(T='company',W1='the')

0.08976377952755905

## 14 Given this, if the word 'the' is found in a name, what is the most likely type?

Movie is the most likely type becaue P(Movie|the) has the highest probability

## 15. Is Ptw(T='movie'|W1='the') the same as Pwt(W1='the'|T='movie') the same? Why or why not?

In [94]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [95]:
Pwt(W1='the', T='movie')

0.09214308527626956

(why or why not? answer here)

They are not the same because P(T|W1) is not equal to P(W1|T).

## 16. Do you think modeling Ptw(T|W1) would be better with a continuous function like a Gaussian? Why or why not?

- Answer in a markdown cell


I do not think modelling Ptw(T|W1) would be better with a continuous function like a Gaussian, because in this case our data set has discrete values. Gaussian is used to model continuous data.